
|  |
| ------------------------------------------------------- | 
| ![Tremplin des sciences](images/tremplinColorSmall.png) | 

Cahier d'exercices pour l'enseignement et l'apprentissage de programmation issu de la collection "Climat et météo tremplin pour l'enseignement des sciences" (PIA IFÉ ENS de Lyon - Météofrance ENM Toulouse). Les sources exécutables sont accessibles sur [la forge github](https://github.com/g-vidal/CahierDeProgrammes); plus d'information sur les [blogs d'accompagnement](http://blog.climatetmeteo.fr/GerardVidal/).  Toutes les ressources sont fournies sous licence [Creative Commons](https://creativecommons.org/licenses/by-nc/4.0/) ![licence : Creative Commons](images/Licence.jpg) 

Auteurs : G. Vidal, C-H. Eyraud, E. le Jan

------------------------------------------------------------

# Comparaison de  3 mesures avec 3 capteurs identiques

Les instruments de mesure de concentration gazeuses de la famille MQ  fournissent un signal analogique c'est pourquoi il faut les convertir en signal numérique pour pouvoir les exploiter avec une RaspberryPI

## Utilisation de convertisseurs Analogique --> digital

Nous avons à notre disposition deux convertisseurs analogique numérique

 * ADS1015 Résolution de 12 bit, Fréquence d'échantillonnage Max :3300 (SPS_128 à SPS_3300)
 * ADS1115 Résolution  de 16bit, Fréquence d'échantillonage Max : 860 (SPS_8 à SPS_860)
dont le fonctionnement est totalement identique. Ce qui les diffférencie est la grandeur privilégiée: précision de l'échantillonnage ou fréquence d'échantillonnage.

Deux modes de fonctionnement sont possibles :
 * en mode différentiel le signal est mesuré par différence entre la broche de sortie et la masse
 * en mode direct le signal est mesuré directement sur la broche de sortie
Nous disposons de 3 capteurs identiques et de 4 broches d'échantillonnage sur le convertisseur, nous ferons deux mesures directes et une mesure par différence. L'exploitation fine de ces éléments requiert le calibrage des capteurs analogiques et le choix des bons paramètres de configuration.

La figure ci-dessous présente l'ensemble du dispositif :

![Convertisseurs vue générale](images/3CapteursTout.jpg)
                        ![licence : Gérard Vidal](images/Licence.jpg)
                        
Le câblage du convertisseurs ne présente pas de difficulté particulière, le point requérant un peu d'attention est le choix de l'adresse qui sera envoyée sur le bus `I2C` car il faut choisir cette adresse parmi les 4 possibles :  `0x49` `0x48` `0x4B` `0x4A`. Le choix se fait en établissant une connexion entre la broche `ADDR` et l'une des 4 broches `VDD` `GND` `SCL` `SDA`. Le tableau ci-dessous indique les équivalences des connexions :

| Broche | Adresse |
|:------ | ------- |
| `VDD`  | `0x49`  |
| `GND`  | `0x48`  |
| `SCL`  | `0x4B`  |
| `SDA`  | `0x4A`  |

les 3 capteurs utilisés sont identiques et permettent de mesurer la qualité de l'air 


L'image suivante montre les connexions au convertisseur analogique numérique ADS 1015 :

![Vue des connexions au convertisseur](images/3CapteursADS1015.jpg)
                        ![licence : Gérard Vidal](images/Licence.jpg)

L'image suivante montre les connexions au convertisseur de niveau 5V-3,3V (Level Shifter) :

![Le level shifter](images/3CapteursLevelShifter.jpg)
                        ![licence : Gérard Vidal](images/Licence.jpg)
                        
Importation des librairies nécessaires à l'utilisation du convertisseur. On n'importe aucune librairie propre aux capteurs de gaz car il s'agit de capteurs `analogiques` qui transmettent une variation de potentiel sur un fil série.

In [1]:
import mraa, upm
from time import sleep
from upm import pyupm_ads1x15 as upm

## Récupération des paramètres d'état de l'échantillonneur

On déclare ici l'échantillonneur et on récupère les valeurs de ses paramètres d'opération (gain, fréquence et mode). 
 * getGain()
 * getSPS()
 * getContinuous()
 
On fixe ensuite ces paramètres pour la manipulation que l'on souhaite effectuer.
 * setGain()
 * setSPS()
 * setContinuous()

In [40]:
gaz = upm.ADS1015(0, 0x49)
gain = gaz.getGain()
sps = gaz.getSPS()
cont = gaz.getContinuous() 
print ("Gain : {0:5d}".format(gain)," SPS : {0:5d}".format(sps), 
       "Continuous : {0:5d}".format(cont))
gaz.setContinuous(True) # Pour ne pas faire de conversion en continu
gaz.setGain(upm.ADS1X15.GAIN_ONE) # Passage à un gain 1024
gaz.setSPS(upm.ADS1015.SPS_250) # Passage à un taux d'échantillonage de 250 Sample Per Second
# Vérification
gain = gaz.getGain()
sps = gaz.getSPS()
cont = gaz.getContinuous() 
print ("\nNouvelles valeurs :")
print ("Gain : {0:5d}".format(gain)," SPS : {0:5d}".format(sps), 
       "Continuous : {0:5d}".format(cont))

Gain :   512  SPS :    96 Continuous :     1

Nouvelles valeurs :
Gain :   512  SPS :    32 Continuous :     1


## Mesure directe des variables qualité de l'air

La première mesure est faite par différence, les deux dernières directement.

In [46]:
air1 = gaz.getSample(upm.ADS1X15.DIFF_0_1)
air2 = gaz.getSample(upm.ADS1X15.SINGLE_2)
air3 = gaz.getSample(upm.ADS1X15.SINGLE_3)
print ("Qualité Air 1 : {0:.2f}\n".format(air1),
       "Qualité Air 2 : {0:.2f}\n".format(air2),
       "Qualité Air 3 : {0:.2f}\n".format(air3))

Qualité Air 1 : 1.90
 Qualité Air 2 : 2.38
 Qualité Air 3 : 1.92



# Série de mesures à l'aide d'une boucle
Dans le groupe suivant on répète 20 fois la mesure en partant de 0 et en allant jusqu'à 19 (inférieur à 20) au pas de 1 par défaut.

In [ ]:
for num in range(0,20):
    air1 = gaz.getSample(upm.ADS1X15.DIFF_0_1)
    air2 = gaz.getSample(upm.ADS1X15.SINGLE_2)
    air3 = gaz.getSample(upm.ADS1X15.SINGLE_3)
    print ("Qualité Air 1 : {0:.2f}\n".format(air1),
       "Qualité Air 2 : {0:.2f}\n".format(air2),
       "Qualité Air 3 : {0:.2f}\n".format(air3))    
    sleep(2)